In [10]:
import pandas as pd
import numpy as np
import nltk
import re
import datetime
import pickle
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import words
from nltk.corpus import stopwords
from contractions import CONTRACTION_MAP
# from stopwords import stop_words
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
def expand_contractions(word):
    """
    This function expands words such as I'll to I will.
    :param word: a single review
    :returns: the expanded words
    """
    expanded = ' '.join([CONTRACTION_MAP[t] if t in CONTRACTION_MAP else t for t in word.split(" ") ])
    return expanded

def get_wordnet_pos(word):
    """
    This function gets the wordnet postag of each words.
    :param word: word in each review texts
    :returns: the postag of each word 
    """
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N":wordnet.NOUN, "V":wordnet.VERB, "R":wordnet.ADV}
    
    return tag_dict.get(tag,wordnet.NOUN)
    
def preprocessing_text(text):
    """
    This function preprocesses the review texts by performing contractions, removing numbers and
    punctuations, make all the characters into lowercase, tokenization, lemmatization as well as removing stopwords.
    :param text: a single text review
    :returns: a list of preprocessed words
    """
    #contractions
    expanded_text=expand_contractions(text)
    #remove numbers
    numbers_removed = re.sub(r'\d+','',expanded_text)
    #remove punctuation
    punct_removed = re.sub(r'[^\w\s]','',numbers_removed)
    #tokenization
    tokens = nltk.word_tokenize(punct_removed.lower())
    
    #remove stop words and lemmatization
    lem_words = []
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    for word in tokens:
        temp_word = lemmatizer.lemmatize(word,get_wordnet_pos(word))
        if  temp_word not in stop_words:
            lem_words.append(temp_word)

    return lem_words

def calculate_tfidf(text):
    """
    This function calculates the tf-idf values for the review text that user inputted.
    :params text: review text that user inputted
    :returns: the tfidf values calculated
    """
    def dummy_func(docs):
        """
        This is just a dummy function.
        """
        return docs
    
    transformer = TfidfTransformer()
    loaded_vec = TfidfVectorizer(analyzer='word',tokenizer=dummy_func, preprocessor=dummy_func, token_pattern=None,vocabulary = pickle.load(open("feature.pkl", "rb")))
    tfidf = transformer.fit_transform(loaded_vec.fit_transform(text))
    
    print(tfidf.shape)
    print(tfidf)
       
    return tfidf



In [30]:
'''
This is the basic user interface used to predict rating from a text review using Tkinter.
'''

from tkinter import *
from tkinter import font
from tkinter.scrolledtext import ScrolledText

        
def reset():
    '''
    This function resets the review textbox and the predicted ratings textbox.
    '''
    textbox.delete(1.0,END)    #deletes the text inside the textbox (user input)
    result.config(state='normal')  #allows the output textbox to be modified
    result.delete(0,END)     #deletes the text inside the output textbox
    result.config(state='disabled')  #disable the output textbox to be modified

    
def predict():
    '''
    This function displays the predicted rating.
    '''
    result.config(state='normal')   #allows the output textbox to be modified
    result.delete(0,END)      #deletes the text inside the output textbox
    
    user_review = textbox.get(1.0,END) #gets the review text inputted by user
    
    rev_list = []
    Xx = preprocessing_text(user_review)   #preprocesses the review text inputted by user
    rev_list.append(Xx)     #appends the preprocessed review text into the list. This is to ensure that all the words are
                                #considered as a review for tf-idf calculation
    tfidf = calculate_tfidf(rev_list)  #calculates the tf-idf values
    
    loaded_model = pickle.load(open('finalized_model.sav', 'rb'))   #loads the trained model
    prediction = loaded_model.predict(tfidf)    #predicts the rating
    
    
    for rating in prediction:
        result.insert(END,rating)    #outputs the predicted rating
        
    result.config(state='disabled')   #disable the output textbox to be modified


root = Tk()
root.geometry("800x400")    #Sets the size of window
root.configure(background='#DEFFF0')   #Sets the background colour

root.title('Ratings Prediction')  #Sets the title of the root

#Displays the title
main_title = Label(root,text="Ratings Prediction",font=("Elephant",30),bg='#DEFFF0')
main_title.pack(fill=BOTH, expand=0)
#To underline the main_title
f = font.Font(main_title, main_title.cget("font"))
f.configure(underline=True)
main_title.configure(font=f)

#Displays text asking user to enter a review text
text = Label(root, text="Enter a review text: ",font=("Times New Roman",15),bg='#DEFFF0')
text.pack(fill=BOTH, expand=0)

#Displays the textbox where user will input the review text
textbox=ScrolledText(root,height=5,width=40,font=("Times New Roman",13))
textbox.pack(fill=Y,expand=0)

#Create a frame to put the buttons in
frame = Frame(root, height="200", width="200", bg="#DEFFF0",borderwidth = 13)
frame.pack()

#Create a button named "Predict" to predict the ratings
pred_button = Button(frame,text="Predict",font=("Times New Roman",11),command = predict,bg='black',fg='white',height=2,width=10)
pred_button.pack(padx=50, side=LEFT)

#Create a button to clear the review text that user inputted
clear_button = Button(frame,text="Clear Review Text",font=("Times New Roman",11),bg='black',fg='white',command=lambda: textbox.delete(1.0,END),height=2,width=20)
clear_button.pack(padx=50,side=LEFT)

#Create a button to reset the review text that user inputted as well as the displayed rating
reset_button = Button(frame,text="Reset",font=("Times New Roman",11),command=reset,bg='black',fg='white',height=2,width=10)
reset_button.pack(padx=50, side=LEFT)

#Displays the text 
text2 = Label(root,text="The rating for this review is: ",font=("Times New Roman",15),bg='#DEFFF0')
text2.pack(padx=5,expand=0)
  
#Displays the predicted rating
result = Entry(root,font=("Times New Roman",13),bg='white')
result.config(state='disabled',disabledbackground='white',disabledforeground='black',justify=CENTER)
result.pack(fill=Y,expand=0)


root.mainloop() 


(1, 28914)

